<a href="https://colab.research.google.com/github/jpatra85/ColabTF_EDU/blob/master/Transformer_Latest_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_port = pd.read_excel("/content/drive/MyDrive/October 2022.xlsx")
df_port.head()

,PortGeoId,PortName,Country,Latitude,Longitude,Old_id,Port_id,Status
0,1,Skaramanga,GR,38.002000,23.589667,PO7898,213848,Sub-Port
1,3,Phaleron Bay,GR,37.942746,23.672563,PO7902,213862,Sub-Port
2,4,Kos,GR,36.878691,27.279368,PO3599,21698,Open
3,6,Zante,GR,37.779669,20.896372,PO1717,22902,Open
4,7,Orsova,R,44.725376,22.416888,PO6518,23502,Open


In [ ]:
df_port.Country.nunique()

204

In [3]:
data = pd.read_csv("/content/drive/MyDrive/hex_grid_sequence_agg_col_vals_df_v2.1_2024-05-31-2.csv")
data.head()

,voyage_id,IHSLRorIMONo,load_port,discharge_port,vessel_dwt,ShiptypeGroup,Operator,vessel_age,masked_hex_res3,col_values
0,L-8307789-2019128-15926-2019122-214424,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29741', 'lab29740', 'lab29581', 'lab29739...","[array([26.043218707806474, 56.705781135637366..."
1,L-8307789-2019225-105936-2019220-113512,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29740', 'lab29739', 'lab30049', 'lab30048...","[array([26.484344867459626, 55.60015016229266,..."
2,L-8307789-2019310-45931-201933-13520,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29740', 'lab30048', 'lab30353', 'lab30516...","[array([26.484344867459626, 55.60015016229266,..."
3,L-8307789-2019324-215726-2019318-104941,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29741', 'lab29740', 'lab29581', 'lab29739...","[array([26.043218707806474, 56.705781135637366..."
4,L-8308147-2019926-15735-2019924-95859,8308147,Port of South Louisiana,Puerto Cortes,42913,"General Cargo =>30,000dwt",SMT Shipping Cyprus Ltd,40,"['lab30768', 'lab30600', 'lab30298', 'lab29985...","[array([30.014169956560348, -90.4982927987873,..."


In [4]:
data_mrg = data.merge(df_port, left_on='discharge_port', right_on='PortName', how='inner')
data_mrg.shape, data.shape

((40133, 18), (40133, 10))

In [5]:
data = data_mrg.copy()

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [7]:
#Convert masked_seq from string to list of strings
data['masked_seq'] = data.masked_hex_res3.apply(eval)

In [8]:
data['masked_seq_len'] = data['masked_seq'].apply(len)
data = data[data['masked_seq_len'] > 20].copy()

In [12]:
# Function to create the samples at different levels
def create_samples(seq, level):
    return seq[:int(len(seq) * level)]

# Create a new DataFrame with three times the rows
sample_levels = [0.25]
expanded_data = pd.DataFrame({
    'masked_seq': [seq for seq in data['masked_seq'] for _ in sample_levels],
    'load_port': [seq for seq in data['load_port'] for _ in sample_levels],
    'discharge_port': [seq for seq in data['discharge_port'] for _ in sample_levels],
    'Country': [seq for seq in data['Country'] for _ in sample_levels],
    'sample_level': [level for _ in range(len(data)) for level in sample_levels]
})

# Apply the sampling function to create the new sampled sequences
expanded_data['masked_seq_sampled'] = expanded_data.apply(lambda row: create_samples(row['masked_seq'], row['sample_level']), axis=1)

# Drop the 'sample_level' column as it's no longer needed
expanded_data = expanded_data.drop(columns=['sample_level'])


In [31]:
expanded_data = expanded_data[expanded_data['Country'] != 'NC']

In [36]:
expanded_data.groupby(['load_port', 'discharge_port']).size()[expanded_data.groupby(['load_port', 'discharge_port']).size() < 10].reset_index()

,load_port,discharge_port,0
0,Mina Saqr,Porbandar,3
1,Mina Saqr,Port Okha,1
2,Samarinda,Masinloc,6
3,Samarinda,Sual Port,4


In [37]:
expanded_data = expanded_data[expanded_data['discharge_port'] != 'Porbandar']
expanded_data = expanded_data[expanded_data['discharge_port'] != 'Port Okha']
expanded_data = expanded_data[expanded_data['discharge_port'] != 'Masinloc']
expanded_data = expanded_data[expanded_data['discharge_port'] != 'Sual Port']

In [29]:
# sequences = tokenizer.texts_to_sequences(expanded_data['masked_seq_sampled'])

# # Pad the sequences
# max_len = 208
# x_data = pad_sequences(sequences, maxlen=max_len, padding='post')

# # Encode the class labels
# label_encoder = LabelEncoder()
# y_data = label_encoder.fit_transform(expanded_data['Country'])
# num_classes = len(label_encoder.classes_)

# # Stratified sampling
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, stratify=expanded_data[['load_port', 'discharge_port']], random_state=42)

In [38]:
#Convert masked_seq from string to list of strings
#data['masked_seq'] = data.masked_hex_res3.apply(eval)

# Tokenize the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(expanded_data['masked_seq_sampled'])

sequences = tokenizer.texts_to_sequences(expanded_data['masked_seq_sampled'])

# Pad the sequences
max_len = 86
x_data = pad_sequences(sequences, maxlen=max_len, padding='post')

# Encode the class labels
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(expanded_data['Country'])
num_classes = len(label_encoder.classes_)

# Stratified sampling
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, stratify=expanded_data[['load_port', 'discharge_port']], random_state=42)


# Load custom embeddings (example with random embeddings)
vocab_size = len(tokenizer.word_index) + 1
embed_dim = 64
embedding_matrix = np.random.rand(vocab_size, embed_dim)  # Replace this with actual embeddings

# Multi-Head Attention
class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

# Feed-Forward Network
class FeedForwardNetwork(Layer):
    def __init__(self, dff, d_model):
        super(FeedForwardNetwork, self).__init__()
        self.dense1 = Dense(dff, activation='relu')
        self.dense2 = Dense(d_model)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Transformer Encoder Layer
class TransformerEncoderLayer(Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = FeedForwardNetwork(feed_forward_dim, embed_dim)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.mha(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build Transformer Encoder Model
def build_transformer_encoder(vocab_size, max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes):
    inputs = tf.keras.layers.Input(shape=(max_len,))
    embedding_layer = Embedding(vocab_size, embed_dim, weights=[embedding_matrix], trainable=False)(inputs)

    x = embedding_layer
    for _ in range(num_layers):
        x = TransformerEncoderLayer(embed_dim, num_heads, feed_forward_dim)(x, training=True)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    # x = Dense(512, activation='relu')(x)  # First additional FFN layer
    # x = Dropout(0.1)(x)
    # x = Dense(512, activation='relu')(x)  # Second additional FFN layer
    # x = Dropout(0.25)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)


# Check for GPUs and set memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
    except RuntimeError as e:
        print(e)

# Parameters
embed_dim = 64
num_heads = 8
feed_forward_dim = 256
num_layers = 2

# Build the model
model = build_transformer_encoder(vocab_size, max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation
history = model.fit(x_train , y_train, epochs=12, batch_size=256, validation_data=(x_val, y_val))

# Function to predict and compare actual vs. predicted labels
def predict_and_compare(model, x_data, y_data, label_encoder):
    y_pred = model.predict(x_data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_data

    y_true_labels = label_encoder.inverse_transform(y_true_classes)
    y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

    for i in range(len(y_true_labels)):
        print(f"Actual: {y_true_labels[i]}, Predicted: {y_pred_labels[i]}")

# Predict and compare on validation set
#predict_and_compare(model, x_val, y_val, label_encoder)

1 Physical GPUs, 1 Logical GPUs
Epoch 1/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 23s 107ms/step - accuracy: 0.4110 - loss: 2.4876 - val_accuracy: 0.5863 - val_loss: 1.5119
Epoch 2/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.6371 - loss: 1.3058 - val_accuracy: 0.6929 - val_loss: 0.9519
Epoch 3/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.6946 - loss: 0.9501 - val_accuracy: 0.7081 - val_loss: 0.8729
Epoch 4/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7068 - loss: 0.8864 - val_accuracy: 0.7152 - val_loss: 0.8469
Epoch 5/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7094 - loss: 0.8502 - val_accuracy: 0.7176 - val_loss: 0.8334
Epoch 6/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7163 - loss: 0.8235 - val_accuracy: 0.7157 - val_loss: 0.8186
Epoch 7/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7219 - loss: 0.8022 - val_accuracy: 0.7196 - val_loss: 0.8198
Epoch 8/12
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy

In [40]:
history = model.fit(x_train , y_train, epochs=2, batch_size=256, validation_data=(x_val, y_val))

Epoch 1/2
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7412 - loss: 0.7253 - val_accuracy: 0.7264 - val_loss: 0.7850
Epoch 2/2
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7446 - loss: 0.7197 - val_accuracy: 0.7324 - val_loss: 0.7785


In [41]:
y_pred = model.predict(x_val)
y1 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-1:])
y2 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-2:-1])
y3 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-3:-2])

y_act = label_encoder.inverse_transform(y_val)

df_p = pd.DataFrame([y1,y2,y3, y_act]).T


234/234 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
np.mean(df_p[0] == df_p[3]) + np.mean(df_p[1] == df_p[3]) + np.mean(df_p[2] == df_p[3])

0.9231695890777674

In [ ]:
df_p.to_csv("top3_pred_15_to_25_pct_input.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
top3 = np.argsort(-model.predict(x_val), axis = 1)[:3]

243/243 [==============================] - 2s 9ms/step


In [ ]:
top3

In [ ]:
# Function to predict and compare actual vs. predicted labels
def predict_and_compare(model, x_data, y_data, label_encoder):
    y_pred = model.predict(x_data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_data

    y_true_labels = label_encoder.inverse_transform(y_true_classes)
    y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

    for i in range(len(y_true_labels)):
        print(f"Actual: {y_true_labels[i]}, Predicted: {y_pred_labels[i]}")

# Predict and compare on validation set
#predict_and_compare(model, x_val, y_val, label_encoder)

In [ ]:
y_pred = model.predict(x_val)

y1 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-1:])
y2 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-2:-1])
y3 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-3:-2])
y_act = label_encoder.inverse_transform(y_val)

df_p = pd.DataFrame([y1,y2,y3, y_act]).T


701/701 [==============================] - 4s 6ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
np.sum((df_p[0] == df_p[3]) | (df_p[1] == df_p[3]) | (df_p[2] == df_p[3])) / df_p.shape[0]

0.9078137543484078

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(expanded_data, y_data, test_size=0.2, stratify=expanded_data[['load_port', 'discharge_port']], random_state=42)

In [ ]:
x_val['T1'] = y1
x_val['T2'] = y2
x_val['T3'] = y3

In [ ]:
df_p.to_csv("x_val_with_pred_top3.csv")

In [ ]:
y_pred = np.argmax(model.predict(x_val), axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_true_labels = label_encoder.inverse_transform(y_val)

251/251 [==============================] - 6s 25ms/step


In [ ]:
d = pd.DataFrame([y_true_labels, y_pred_labels == y_true_labels]).T
d[1] = d[1].astype(int)

In [ ]:
grouped = d.groupby(0)[1]
accuracy_count = grouped.sum()
total = grouped.count()
percent = (accuracy_count / total) * 100

# Create the new DataFrame with the calculated statistics
d1 = pd.DataFrame({
    "accuracy_count": accuracy_count,
    "total": total,
    "percent": percent
}).reset_index()

# Display the resulting DataFrame
#(d1.to_csv('model_accuracy_by_port.csv'))

In [ ]:
d1.columns = ['PortName','accuracy_count', 'total', 'percent']

In [ ]:
d2 = d1.merge(df_port, on='PortName', how='left')

In [ ]:
d2

,PortName,accuracy_count,total,percent,PortGeoId,Country,Latitude,Longitude,Old_id,Port_id,Status
0,Acajutla,19,19,100.000000,4269,ES,13.577100,-89.840017,PO3314,29902,Open
1,Alexandria (Egypt),3,23,13.043478,4836,ET,31.190833,29.875333,PO1507,252278,Open
2,Altamira,12,12,100.000000,1803,MEX,22.487587,-97.849151,PO5693,29313,Open
3,Bahodopi,27,27,100.000000,11049,RI,-2.815000,122.163000,PO9764,462646,Open
4,Bandar Imam Khomeini,40,41,97.560976,719,IR,30.430552,49.057585,PO1853,28871,Open
...,...,...,...,...,...,...,...,...,...,...,...
189,Zhangzhou,1,7,14.285714,2439,CHI,24.403824,118.038649,PO6742,12903,Open
190,Zhanjiang,106,117,90.598291,9050,CHI,21.063550,110.458050,PO3766,28775,Open
191,Zhenjiang,27,46,58.695652,2179,CHI,32.219667,119.383333,PO5141,26169,Open
192,Zhoushan,148,199,74.371859,2276,CHI,29.989691,122.119528,PO6033,23224,Open


In [ ]:
d2_group = d2.groupby('Country')[['accuracy_count', 'total']].sum()
d2_group['percent'] = (d2_group['accuracy_count'] / d2_group['total']) * 100

In [ ]:
d2_group.sort_values('percent', ascending=True)

,accuracy_count,total,percent
Country,,,
ET,25,49,51.020408
B,8,14,57.142857
IRQ,6,9,66.666667
RCH,6,9,66.666667
JAP,627,922,68.004338
CHI,2397,3286,72.945831
GB,6,8,75.000000
UA,35,46,76.086957
BR,50,64,78.125000


from matplotlib import pyplot as plt
_df_0['accuracy_count'].plot(kind='hist', bins=20, title='accuracy_count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['total'].plot(kind='hist', bins=20, title='total')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['percent'].plot(kind='hist', bins=20, title='percent')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='accuracy_count', y='total', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['percent']
  ys = series['total']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_6.sort_values('percent', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('percent')
_ = plt.ylabel('total')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['percent']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'percent'}, axis=1)
              .sort_values('percent', ascending=True))
  xs = counted['percent']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('percent', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('percent')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_8['accuracy_count'].plot(kind='line', figsize=(8, 4), title='accuracy_count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['total'].plot(kind='line', figsize=(8, 4), title='total')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['percent'].plot(kind='line', figsize=(8, 4), title='percent')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
#Convert masked_seq from string to list of strings
data['masked_seq'] = data.masked_hex_res3.apply(eval)

# Tokenize the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['masked_seq'])
sequences = tokenizer.texts_to_sequences(data['masked_seq'])


# Pad the sequences
max_len = 400
x_data = pad_sequences(sequences, maxlen=max_len, padding='post')

# Encode the class labels
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(data['discharge_port'])
num_classes = len(label_encoder.classes_)

# Stratified sampling
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, stratify=data[['load_port', 'discharge_port']], random_state=42)

# Load custom embeddings (example with random embeddings)
vocab_size = len(tokenizer.word_index) + 1
embed_dim = 64
embedding_matrix = np.random.rand(vocab_size, embed_dim)  # Replace this with actual embeddings

# Multi-Head Attention
class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

# Feed-Forward Network
class FeedForwardNetwork(Layer):
    def __init__(self, dff, d_model):
        super(FeedForwardNetwork, self).__init__()
        self.dense1 = Dense(dff, activation='relu')
        self.dense2 = Dense(d_model)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Transformer Encoder Layer
class TransformerEncoderLayer(Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = FeedForwardNetwork(feed_forward_dim, embed_dim)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.mha(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build Transformer Encoder Model
def build_transformer_encoder(vocab_size, max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes):
    inputs = tf.keras.layers.Input(shape=(max_len,))
    embedding_layer = Embedding(vocab_size, embed_dim, weights=[embedding_matrix], trainable=False)(inputs)

    x = embedding_layer
    for _ in range(num_layers):
        x = TransformerEncoderLayer(embed_dim, num_heads, feed_forward_dim)(x, training=True)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    # x = Dense(512, activation='relu')(x)  # First additional FFN layer
    # x = Dropout(0.1)(x)
    # x = Dense(512, activation='relu')(x)  # Second additional FFN layer
    # x = Dropout(0.25)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)


# Check for GPUs and set memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
    except RuntimeError as e:
        print(e)

# Parameters
embed_dim = 64
num_heads = 8
feed_forward_dim = 256
num_layers = 2

# Build the model
model = build_transformer_encoder(vocab_size, max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_val, y_val))

# Function to predict and compare actual vs. predicted labels
def predict_and_compare(model, x_data, y_data, label_encoder):

    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_data

    y_true_labels = label_encoder.inverse_transform(y_true_classes)
    y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

    for i in range(len(y_true_labels)):
        print(f"Actual: {y_true_labels[i]}, Predicted: {y_pred_labels[i]}")

# Predict and compare on validation set
predict_and_compare(model, x_val, y_val, label_encoder)

1 Physical GPUs, 1 Logical GPUs
Epoch 1/5
1004/1004 [==============================] - 67s 60ms/step - loss: 3.1702 - accuracy: 0.2692 - val_loss: 1.3768 - val_accuracy: 0.5905
Epoch 2/5
1004/1004 [==============================] - 65s 65ms/step - loss: 1.0445 - accuracy: 0.6593 - val_loss: 0.7851 - val_accuracy: 0.7244
Epoch 3/5
1004/1004 [==============================] - 60s 59ms/step - loss: 0.7389 - accuracy: 0.7425 - val_loss: 0.6552 - val_accuracy: 0.7778
Epoch 4/5
1004/1004 [==============================] - 64s 64ms/step - loss: 0.6272 - accuracy: 0.7779 - val_loss: 0.6033 - val_accuracy: 0.7901
Epoch 5/5
1004/1004 [==============================] - 59s 59ms/step - loss: 0.5626 - accuracy: 0.7976 - val_loss: 0.5449 - val_accuracy: 0.8099


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# Parameters
embed_dim = 64
num_heads = 8
feed_forward_dim = 512
num_layers = 2

# Build the model
model = build_transformer_encoder(vocab_size, max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_val, y_val))


Epoch 1/5
502/502 [==============================] - 65s 117ms/step - loss: 3.4810 - accuracy: 0.2132 - val_loss: 1.8222 - val_accuracy: 0.4521
Epoch 2/5
 66/502 [==>...........................] - ETA: 45s - loss: 1.7308 - accuracy: 0.4754

KeyboardInterrupt: 

In [ ]:
data.head()

,voyage_id,IHSLRorIMONo,load_port,discharge_port,vessel_dwt,ShiptypeGroup,Operator,vessel_age,masked_hex_res3,col_values,masked_seq
0,L-8307789-2019128-15926-2019122-214424,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29741', 'lab29740', 'lab29581', 'lab29739...","[array([26.043218707806474, 56.705781135637366...","[lab29741, lab29740, lab29581, lab29739, lab29..."
1,L-8307789-2019225-105936-2019220-113512,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29740', 'lab29739', 'lab30049', 'lab30048...","[array([26.484344867459626, 55.60015016229266,...","[lab29740, lab29739, lab30049, lab30048, lab30..."
2,L-8307789-2019310-45931-201933-13520,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29740', 'lab30048', 'lab30353', 'lab30516...","[array([26.484344867459626, 55.60015016229266,...","[lab29740, lab30048, lab30353, lab30516, lab30..."
3,L-8307789-2019324-215726-2019318-104941,8307789,Mina Saqr,Shuaiba,41093,Bulk Carrier-Handymax,Al Khalejia Aggregates FZE,40,"['lab29741', 'lab29740', 'lab29581', 'lab29739...","[array([26.043218707806474, 56.705781135637366...","[lab29741, lab29740, lab29581, lab29739, lab29..."
4,L-8308147-2019926-15735-2019924-95859,8308147,Port of South Louisiana,Puerto Cortes,42913,"General Cargo =>30,000dwt",SMT Shipping Cyprus Ltd,40,"['lab30768', 'lab30600', 'lab30298', 'lab29985...","[array([30.014169956560348, -90.4982927987873,...","[lab30768, lab30600, lab30298, lab29985, lab28..."


In [ ]:
data.col_values.head(1).values

array(["[array([26.043218707806474, 56.705781135637366, 6.4, 355.0, 11.4, '41093',\n       40, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,\n       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object), array([26.484344867459626, 55.60015016229266, 1.25152, 219.90909, 11.4,\n       '41093', 40, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object), array([25.907257461408154, 54.682303553203674, 8.6625, 271.125, 11.4,\n       '41093', 40, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object), array([26.329731631467382, 53.58072502364251, 9.28, 302.2, 11.4, '41093',\n       40, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,\n       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object), array([26.742714344610498, 52.47390780080044, 8.85714, 298.28571, 11.4,\n       '41093', 40, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n       0, 1, 1, 0, 

In [ ]:
import re
f = (lambda x: re.findall(r'\d+\.\d+|\d+', x))
data['embb_seq'] = data.col_values.apply(f)
data[''] = data.embb_seq.apply(lambda x: [float(value) if '.' in value else int(value) for value in x])
data['embd_seq_len'] = data['embb_seq'].apply(len) / 38
data_filtered = data[data['embd_seq_len'] > 20].copy()

In [ ]:
data_filtered.head()

,voyage_id,IHSLRorIMONo,load_port,discharge_port,vessel_dwt,ShiptypeGroup,Operator,vessel_age,masked_hex_res3,col_values,embb_seq,embd_seq_len
29,L-8309397-20231019-112030-202399-223939,8309397,Mina Saqr,Male,43952,Bulk Carrier-Handymax,SMT Shipmanagement & Trans-POL,40,"['lab29740', 'lab29741', 'lab29582', 'lab29265...","[array([26.484344867459626, 55.60015016229266,...","[26.484344867459626, 55.60015016229266, 10.7, ...",36.0
30,L-8309397-2023123-10028-20231121-124050,8309397,Mina Saqr,Male,43952,Bulk Carrier-Handymax,SMT Shipmanagement & Trans-POL,40,"['lab29740', 'lab29741', 'lab29423', 'lab29265...","[array([26.484344867459626, 55.60015016229266,...","[26.484344867459626, 55.60015016229266, 10.233...",35.0
32,L-8313702-20231026-6177-2023826-42111,8313702,Mina Saqr,Male,42497,"General Cargo =>30,000dwt",SMT Shipping Cyprus Ltd,38,"['lab29740', 'lab29741', 'lab30052', 'lab29741...","[array([26.484344867459626, 55.60015016229266,...","[26.484344867459626, 55.60015016229266, 9.75, ...",37.0
70,L-8420787-2021122-32825-20201227-231148,8420787,Santos,Vlissingen,43712,"General Cargo =>30,000dwt",G2 Ocean AS,38,"['lab12262', 'lab11928', 'lab12103', 'lab12263...","[array([-23.709493111190206, -46.6334850322877...","[23.709493111190206, 46.63348503228772, 10.05,...",105.0
73,L-8819225-20231026-35717-20231017-72211,8819225,Taboneo,Koh Si Chang Transhipment Area,68433,Bulk Carrier-Pmax/Kamsarmax,Highland Maritime Co Ltd,35,"['lab19285', 'lab18922', 'lab18921', 'lab19094...","[array([-3.5303940948620927, 114.5435614826438...","[3.5303940948620927, 114.54356148264388, 9.0, ...",32.0


In [ ]:
#data_filtered['sample_size'] = data_filtered.embb_seq.apply(lambda x: int(np.random.uniform(0.3, 0.75) * len(x)/ 38) * 38)
data_filtered['sample_size'] = data_filtered.embb_seq.apply(lambda x: int(0.6 * len(x)/ 38) * 38)

In [ ]:
data_filtered['sample_size'].head()

4      342
29     646
30     608
32     532
70    1862
Name: sample_size, dtype: int64

In [ ]:
# Function to sample a percentage of a sequence
def sample_sequence(seq):
    sample_size = int(0.6 * len(seq)/ 38) * 38
    return seq[:sample_size]

# Sample 30% to 75% of each sequence
data_filtered['sampled_seq'] = data_filtered['embb_seq'].apply(lambda x: sample_sequence(x))


In [ ]:
data_filtered['sampled_seq'] .head().apply(lambda x: len(x) / 38 )

29    21.0
30    21.0
32    22.0
70    63.0
73    19.0
Name: sampled_seq, dtype: float64

In [ ]:
import tensorflow as tf
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = data_filtered['sampled_seq'].apply(len).max()

In [ ]:
max_len/38

208.0

In [ ]:
# Pad the sequences to the same length
max_len = max(data_filtered['sampled_seq'].apply(len))
padded_sequences = pad_sequences(data_filtered['sampled_seq'].tolist(), maxlen=max_len, padding='post', dtype='float32')

# Convert the padded sequences to a tensor
tensor = tf.convert_to_tensor(padded_sequences)

In [ ]:
tensor_reshaped = tf.reshape(tensor, (tensor.shape[0], int(max_len / 38), 38))

In [ ]:
tensor_reshaped = tf.pad(tensor_reshaped, [[0, 0], [0, 0], [0, 2]])

In [ ]:
#tensor_reshaped = tensor_reshaped[:, 1::2, :]
#tensor_reshaped.shape

TensorShape([37370, 104, 40])

In [ ]:
import joblib
joblib.dump(tensor_numpy, 'drive/MyDrive/tensor.joblib')


['drive/MyDrive/tensor.joblib']

In [ ]:
joblib.dump(data_filtered, 'drive/MyDrive/data_filtered.joblib')

['drive/MyDrive/data_filtered.joblib']

In [ ]:
# Encode the class labels
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(data_filtered['Country'])
num_classes = len(label_encoder.classes_)

In [ ]:
tensor_reshaped.shape

TensorShape([37370, 104, 40])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder

# Multi-Head Attention
class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

# Feed-Forward Network
class FeedForwardNetwork(Layer):
    def __init__(self, dff, d_model):
        super(FeedForwardNetwork, self).__init__()
        self.dense1 = Dense(dff, activation='relu')
        self.dense2 = Dense(d_model)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Transformer Encoder Layer
class TransformerEncoderLayer(Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = FeedForwardNetwork(feed_forward_dim, embed_dim)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.mha(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build Transformer Encoder Model
def build_transformer_encoder(max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes):
    inputs = Input(shape=(max_len, embed_dim))  # Change input shape to match precomputed embeddings
    x = inputs
    for _ in range(num_layers):
        x = TransformerEncoderLayer(embed_dim, num_heads, feed_forward_dim)(x, training=True)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)

# Encode the class labels
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(data_filtered['Country'])
num_classes = len(label_encoder.classes_)

# Parameters
embed_dim = 40  # Updated to match the precomputed embedding dimension
num_heads = 8
feed_forward_dim = 256
num_layers = 2
actual_max_len = 208  # Update based on the maximum sequence length of your precomputed embeddings

# Build the model
model = build_transformer_encoder(actual_max_len, embed_dim, num_heads, feed_forward_dim, num_layers, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Precomputed embeddings
# Assuming tensor contains the precomputed embeddings
x_train_embedded = tensor_reshaped  # Use the precomputed embeddings

# Train the model with validation split
history = model.fit(x_train_embedded, y_data, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
935/935 [==============================] - 26s 22ms/step - loss: 2.0821 - accuracy: 0.4557 - val_loss: 2.6984 - val_accuracy: 0.4712
Epoch 2/10
935/935 [==============================] - 20s 21ms/step - loss: 1.8867 - accuracy: 0.4730 - val_loss: 2.9812 - val_accuracy: 0.4684
Epoch 3/10
935/935 [==============================] - 20s 21ms/step - loss: 1.8482 - accuracy: 0.4747 - val_loss: 3.0584 - val_accuracy: 0.4742
Epoch 4/10
935/935 [==============================] - 20s 22ms/step - loss: 1.8230 - accuracy: 0.4773 - val_loss: 3.1728 - val_accuracy: 0.4470
Epoch 5/10
935/935 [==============================] - 20s 21ms/step - loss: 1.8004 - accuracy: 0.4810 - val_loss: 3.1520 - val_accuracy: 0.4806
Epoch 6/10
935/935 [==============================] - 20s 21ms/step - loss: 1.4329 - accuracy: 0.5720 - val_loss: 2.8273 - val_accuracy: 0.5522
Epoch 7/10
935/935 [==============================] - 20s 21ms/step - loss: 1.2248 - accuracy: 0.6179 - val_loss: 2.9589 - val_accuracy:

In [ ]:
history = model.fit(x_train_embedded, y_data, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
468/468 [==============================] - 17s 36ms/step - loss: 0.8689 - accuracy: 0.7184 - val_loss: 3.0538 - val_accuracy: 0.5931
Epoch 2/5
468/468 [==============================] - 17s 36ms/step - loss: 0.8463 - accuracy: 0.7256 - val_loss: 3.2006 - val_accuracy: 0.5100
Epoch 3/5
468/468 [==============================] - 17s 36ms/step - loss: 0.8205 - accuracy: 0.7338 - val_loss: 3.2139 - val_accuracy: 0.5851
Epoch 4/5
468/468 [==============================] - 17s 36ms/step - loss: 0.7968 - accuracy: 0.7408 - val_loss: 3.3390 - val_accuracy: 0.5541
Epoch 5/5
468/468 [==============================] - 17s 36ms/step - loss: 0.7722 - accuracy: 0.7485 - val_loss: 3.3263 - val_accuracy: 0.5995


In [ ]:
# Function to predict and compare actual vs. predicted labels
def predict_and_compare(model, x_data, y_data, label_encoder):
    y_pred = model.predict(x_data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_data

    y_true_labels = label_encoder.inverse_transform(y_true_classes)
    y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

    for i in range(len(y_true_labels)):
        print(f"Actual: {y_true_labels[i]}, Predicted: {y_pred_labels[i]}")

# Predict and compare on validation set
#predict_and_compare(model, x_val, y_val, label_encoder)

1168/1168 [==============================] - 10s 9ms/step


In [ ]:
y_pred = model.predict(x_val)
y1 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-1:])
y2 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-2:-1])
y3 = label_encoder.inverse_transform(np.argsort(y_pred, axis=1)[:,-3:-2])

y_act = label_encoder.inverse_transform(y_val)

df_p = pd.DataFrame([y1,y2,y3, y_act]).T


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
np.sum((df_p[0] == df_p[3]) | (df_p[1] == df_p[3]) | (df_p[2] == df_p[3])) / df_p.shape[0]

0.8948086700561948

In [ ]:
np.sum((df_p[0] == df_p[3])) / df_p.shape[0]

0.6881723307465881